In [1]:
import torch
from torch import nn
import csv
import os
train_on_gpu=True

In [2]:
import re
from collections import Counter
import numpy as np
import csv

In [3]:
import helper

In [4]:
f=open("ieeesite_data_final.csv","r")
read=csv.reader(f)
data_1=[]
for i in read:
    if i :
        data_1.append(i)

In [5]:
 data_1[:20]

[['yo', 'gen_hi'],
 ['Heya', 'gen_hi'],
 ['Namaste', 'gen_hi'],
 ['Hey', 'gen_hi'],
 ['Great see to you!', 'gen_hi'],
 ['Ssup', 'gen_hi'],
 ['Whatup', 'gen_hi'],
 ['Yo', 'gen_hi'],
 ['hello', 'gen_hi'],
 ['Howdy', 'gen_hi'],
 ['There hey', 'gen_hi'],
 ['There hey!', 'gen_hi'],
 ['! hello', 'gen_hi'],
 ['There hey', 'gen_hi'],
 ['Buddy hey!', 'gen_hi'],
 ['Hi support need', 'gen_hi'],
 ['Hii', 'gen_hi'],
 ['Heyo', 'gen_hi'],
 ['! hello', 'gen_hi'],
 ['There hello!', 'gen_hi']]

In [8]:
def clean(o):
    cleaned=[]
    for i in o:
        if i and i[0]!='' and i[0].count("nan")==0:
            cleaned.append(i)
    return cleaned

In [9]:
clean(data_1)[:20]

[['yo', 'gen_hi'],
 ['Heya', 'gen_hi'],
 ['Namaste', 'gen_hi'],
 ['Hey', 'gen_hi'],
 ['Great see to you!', 'gen_hi'],
 ['Ssup', 'gen_hi'],
 ['Whatup', 'gen_hi'],
 ['Yo', 'gen_hi'],
 ['hello', 'gen_hi'],
 ['Howdy', 'gen_hi'],
 ['There hey', 'gen_hi'],
 ['There hey!', 'gen_hi'],
 ['! hello', 'gen_hi'],
 ['There hey', 'gen_hi'],
 ['Buddy hey!', 'gen_hi'],
 ['Hi support need', 'gen_hi'],
 ['Hii', 'gen_hi'],
 ['Heyo', 'gen_hi'],
 ['! hello', 'gen_hi'],
 ['There hello!', 'gen_hi']]

In [10]:
data=clean(data_1)
data[:20]

[['yo', 'gen_hi'],
 ['Heya', 'gen_hi'],
 ['Namaste', 'gen_hi'],
 ['Hey', 'gen_hi'],
 ['Great see to you!', 'gen_hi'],
 ['Ssup', 'gen_hi'],
 ['Whatup', 'gen_hi'],
 ['Yo', 'gen_hi'],
 ['hello', 'gen_hi'],
 ['Howdy', 'gen_hi'],
 ['There hey', 'gen_hi'],
 ['There hey!', 'gen_hi'],
 ['! hello', 'gen_hi'],
 ['There hey', 'gen_hi'],
 ['Buddy hey!', 'gen_hi'],
 ['Hi support need', 'gen_hi'],
 ['Hii', 'gen_hi'],
 ['Heyo', 'gen_hi'],
 ['! hello', 'gen_hi'],
 ['There hello!', 'gen_hi']]

In [11]:
questions=[]
labels=[]
words=[]
for i in data:
    questions.append(i[0])
    labels.append(i[1])
    for j in i[0].split():
        words.append(j)
y=labels

In [18]:
words=list(set(words))
questions=questions

labels=list(set(labels))
dead_words=[]

In [19]:
for i in words:
    try:
        print("",end="")
    except KeyError:
        dead_words.append(i)

In [20]:
len(dead_words)

0

In [21]:
def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', '')
    text = text.replace(',','')
    text = text.replace('"', '')
    text = text.replace(';', '')
    text = text.replace('!', '')
    text = text.replace('?', '')
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('--', '')
    text = text.replace('?', '')
    text = text.replace('\n', '')
    text = text.replace(':', '')
    
    return text

In [22]:
words[:20]

['my,',
 'pen',
 'serve!',
 'onto',
 'permanent',
 "this's",
 'asian',
 'herb',
 'project?',
 'cognise',
 'theatre',
 'large',
 'amazing',
 'adaptation',
 'international',
 'Ssup',
 'develop',
 'who?',
 'yourself',
 'back']

In [23]:
cleaned_words=[]
for i in words:
    if not i in dead_words:
        cleaned_words.append(preprocess(i))

In [24]:
word_to_int={}
int_to_word={}
for i,j in enumerate(cleaned_words,1):
    word_to_int.update({j:i})
for j,i in word_to_int.items():
    int_to_word.update({i:j})

In [25]:
label_to_int={}
int_to_label={}
for i,j in enumerate(labels):
    int_to_label.update({i:j})
for i,j in int_to_label.items():
    label_to_int.update({j:i})

In [26]:
print(len(questions),len(y))

14341 14341


In [27]:
def pad_features(reviews_ints, seq_length):
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [28]:
print(len(questions),len(y))

14341 14341


In [29]:
questions_int=[]
question_int=[]
y_int=[]
for i,l in zip(questions,y):
    question_int=[]
    for j in i.split():
        if j in cleaned_words:
            question_int.append(word_to_int[j])
    if len(question_int):
        questions_int.append(question_int)
        y_int.append(label_to_int[l])
        

In [30]:
print(len(y_int),len(questions_int))

14018 14018


In [31]:
questions_int[:20]

[[2877],
 [3516, 3534],
 [4132],
 [3078],
 [4132],
 [3078],
 [3164, 3138],
 [4132],
 [3359, 3499, 2146, 2922],
 [2922, 3499, 2146],
 [4103, 2985, 4118],
 [2922, 3499, 4110, 4010],
 [4010],
 [4029, 2704],
 [4118, 4029],
 [4029, 4118, 3505],
 [1862],
 [3505, 3505],
 [2294, 2516],
 [4103, 2985, 4118]]

In [32]:
questions_int=np.array(questions_int)

C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [33]:
questions_int

array([list([2877]), list([3516, 3534]), list([4132]), ...,
       list([2890, 1711, 3812]),
       list([3924, 4118, 3720, 2409, 269, 4104, 3383]),
       list([2471, 3194, 3720, 666, 3970])], dtype=object)

In [34]:
type(questions_int)

numpy.ndarray

In [35]:
seq_length = 10
features = pad_features(questions_int, seq_length=seq_length)
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."


print(features[:20])

[[   0    0    0    0    0    0    0    0    0 2877]
 [   0    0    0    0    0    0    0    0 3516 3534]
 [   0    0    0    0    0    0    0    0    0 4132]
 [   0    0    0    0    0    0    0    0    0 3078]
 [   0    0    0    0    0    0    0    0    0 4132]
 [   0    0    0    0    0    0    0    0    0 3078]
 [   0    0    0    0    0    0    0    0 3164 3138]
 [   0    0    0    0    0    0    0    0    0 4132]
 [   0    0    0    0    0    0 3359 3499 2146 2922]
 [   0    0    0    0    0    0    0 2922 3499 2146]
 [   0    0    0    0    0    0    0 4103 2985 4118]
 [   0    0    0    0    0    0 2922 3499 4110 4010]
 [   0    0    0    0    0    0    0    0    0 4010]
 [   0    0    0    0    0    0    0    0 4029 2704]
 [   0    0    0    0    0    0    0    0 4118 4029]
 [   0    0    0    0    0    0    0 4029 4118 3505]
 [   0    0    0    0    0    0    0    0    0 1862]
 [   0    0    0    0    0    0    0    0 3505 3505]
 [   0    0    0    0    0    0    0    0 2294

In [36]:
len(labels)

27

In [37]:
split_frac = 0.9
master_intent=y_int
## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = master_intent[:split_idx], master_intent[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")

print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(12616, 10) 
Validation set: 	(701, 10) 
Test set: 		(701, 10)


In [38]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.Tensor(train_x), torch.Tensor(train_y))
valid_data = TensorDataset(torch.Tensor(val_x), torch.Tensor(val_y))
test_data = TensorDataset(torch.Tensor(test_x), torch.Tensor(test_y))

# dataloaders
batch_size = 1

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [39]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([1, 10])
Sample input: 
 tensor([[   0.,    0.,    0.,    0.,    0.,    0., 2792., 2294.,  469., 3949.]])

Sample label size:  torch.Size([1])
Sample label: 
 tensor([22.])


****Model Declaration****

In [40]:
train_on_gpu=torch.cuda.is_available()

In [41]:
class Semantic_Classifier(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(Semantic_Classifier, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim*2, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, hidden):
       
        batch_size = x.size(0)
        embeds = self.embedding(x)
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        out = self.fc(torch.cat((torch.squeeze(hidden[0]),torch.squeeze(hidden[1])),0))
       
        sig_out = self.sig(out)
        sig_out = sig_out.view(batch_size, -1)
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        
        if (status):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden


In [42]:
len(cleaned_words)

4162

In [43]:

vocab_size = len(cleaned_words)+1
output_size = 27
print(vocab_size,output_size)
embedding_dim = 300
hidden_dim = 256
n_layers = 1

# net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net_1=Semantic_Classifier(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net_1)


4163 27
Semantic_Classifier(
  (embedding): Embedding(4163, 300)
  (lstm): LSTM(300, 256, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=27, bias=True)
  (sig): Sigmoid()
)


C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [44]:
lr=0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net_1.parameters(), lr=lr)

In [45]:
status=torch.cuda.is_available()

In [46]:
epochs = 10
counter = 0
print_every = 1000
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net_1.cuda()

net_1.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    net_1.train()
    h = net_1.init_hidden(batch_size)
    

    # batch loop
    for inputs, labels in train_loader:
        counter += 1


        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs=torch.tensor(inputs).to(torch.int64)
            labels=torch.tensor(labels).to(torch.long)

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net_1.zero_grad()

        # get the output from the model
        inputs=torch.tensor(inputs).to(torch.int64)
        labels=torch.tensor(labels).to(torch.long)
        net_1.train()
        output, h = net_1(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net_1.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            net_1.eval()
            val_h = net_1.init_hidden(batch_size)
            val_losses = []
            a=[]
            net_1.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                
                val_h = tuple([each.data for each in val_h])
                inputs=torch.tensor(inputs).to(torch.int64)
                labels=torch.tensor(labels).to(torch.long)
                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                    inputs=torch.tensor(inputs).to(torch.int64)
                    labels=torch.tensor(labels).to(torch.long)

                output, val_h = net_1(inputs, val_h)
                _,index=torch.topk(output,1)
                val_loss = criterion(output, labels)
                a.append(index.item()==labels.item())
            a=100*((a.count(1))/(a.count(0)+a.count(1)))
                

            val_losses.append(val_loss.item())
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.9f}...".format(loss.item()),
                  "Val Accuracy: {:.2f}%...".format(a),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:37: UserWarning: To copy construct from a te

Epoch: 1/10... Step: 1000... Loss: 2.428481817... Val Accuracy: 66.05%... Val Loss: 2.552090
Epoch: 1/10... Step: 2000... Loss: 2.539110899... Val Accuracy: 76.75%... Val Loss: 2.447210
Epoch: 1/10... Step: 3000... Loss: 2.360322475... Val Accuracy: 78.32%... Val Loss: 2.525405
Epoch: 1/10... Step: 4000... Loss: 3.333140612... Val Accuracy: 80.74%... Val Loss: 3.247853
Epoch: 1/10... Step: 5000... Loss: 2.415758610... Val Accuracy: 84.45%... Val Loss: 2.357629
Epoch: 1/10... Step: 6000... Loss: 2.358210087... Val Accuracy: 87.59%... Val Loss: 2.415854
Epoch: 1/10... Step: 7000... Loss: 2.441014290... Val Accuracy: 87.45%... Val Loss: 2.415517
Epoch: 1/10... Step: 8000... Loss: 2.404449940... Val Accuracy: 88.59%... Val Loss: 2.357845
Epoch: 1/10... Step: 9000... Loss: 2.358007193... Val Accuracy: 89.16%... Val Loss: 2.373431
Epoch: 1/10... Step: 10000... Loss: 2.411384821... Val Accuracy: 89.44%... Val Loss: 2.415268
Epoch: 1/10... Step: 11000... Loss: 2.422121286... Val Accuracy: 89.4

Epoch: 8/10... Step: 89000... Loss: 2.357533932... Val Accuracy: 96.58%... Val Loss: 2.357534
Epoch: 8/10... Step: 90000... Loss: 2.357628822... Val Accuracy: 96.72%... Val Loss: 2.357534
Epoch: 8/10... Step: 91000... Loss: 2.357533932... Val Accuracy: 97.15%... Val Loss: 2.357534
Epoch: 8/10... Step: 92000... Loss: 2.357533932... Val Accuracy: 96.72%... Val Loss: 2.357548
Epoch: 8/10... Step: 93000... Loss: 2.357534647... Val Accuracy: 96.86%... Val Loss: 2.357534
Epoch: 8/10... Step: 94000... Loss: 3.167640924... Val Accuracy: 96.86%... Val Loss: 2.357963
Epoch: 8/10... Step: 95000... Loss: 2.415644169... Val Accuracy: 97.00%... Val Loss: 2.357534
Epoch: 8/10... Step: 96000... Loss: 2.357533932... Val Accuracy: 96.86%... Val Loss: 2.357534
Epoch: 8/10... Step: 97000... Loss: 2.400974035... Val Accuracy: 96.72%... Val Loss: 2.357534
Epoch: 8/10... Step: 98000... Loss: 2.357533932... Val Accuracy: 96.86%... Val Loss: 2.357535
Epoch: 8/10... Step: 99000... Loss: 2.357593298... Val Accur

Validation and Test Results are available below

In [47]:
net_1.eval()
val_h = net_1.init_hidden(batch_size)
val_losses = []
a=[]
net_1.eval()
for inputs, labels in test_loader:
        val_h = tuple([each.data for each in val_h])

        if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()
                inputs=torch.tensor(inputs).to(torch.int64)
                labels=torch.tensor(labels).to(torch.long)
                
        output, val_h = net_1(inputs, val_h)
        _,index=torch.topk(output,1)
        a.append(index.item()==labels.item())
        val_loss = criterion(output, labels)
        for i in inputs.squeeze():
            if i:
                print(int_to_word[i.item()],end=" ")
        print('\t\tOUTPUT:',int_to_label[index.item()])
                
        
        
        val_losses.append(val_loss.item())
print( "Val Loss: {:.6f}".format(np.mean(val_losses)),"        Val_Acuuracy:",(a.count(1)/(a.count(0)+a.count(1)))*100)

C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


how are the i doing 		OUTPUT: gen_how_you
whether i want to sponsor 		OUTPUT: ieee_sponsor
yup music that's all 		OUTPUT: gen_thanks
determine how many ieee's in other domains can linked to 		OUTPUT: ieee_other_college
ok what is 		OUTPUT: ieee_techloop
and those're re that cool techloop thing i heard was 		OUTPUT: ieee_techloop
where can i you message you guys 		OUTPUT: ieee_further_contact
who is thee the 		OUTPUT: gen_who_made
can she really have planted a dagger at your shoulder 		OUTPUT: ieee_github
wonder how much do i reach out for collaboration with 		OUTPUT: ieee_collab
how you may contact you 		OUTPUT: ieee_further_contact
does ieee ignition cause 		OUTPUT: ieee_what_does
what matter if something i wanna be a 		OUTPUT: ieee_be_part_who
where down can i put 		OUTPUT: ieee_social_media
can you not be of no practical help aboard 		OUTPUT: gen_help
you wouldnt wish her to be a witch long would 		OUTPUT: gen_bot
peace is for your creator 		OUTPUT: gen_who_made
you say acting like 

how long must i collaborate with those dumb guys 		OUTPUT: ieee_collab
what'what s ieee'll always need 		OUTPUT: ieee_sponsor
what domains do you actually carry 		OUTPUT: ieee_domains
funny what exactly really is 		OUTPUT: ieee_techloop
it 		OUTPUT: ieee_how_long
how can you still save 		OUTPUT: gen_who_you
i all smiled thank thee 		OUTPUT: gen_how_you
is there any way anywhere to contact ieee representatives on 		OUTPUT: ieee_social_media
which storage separation technologies would do you people decide 		OUTPUT: ieee_domains
how can help i get more info 		OUTPUT: ieee_further_contact
how do i get things in touch with without 		OUTPUT: ieee_further_contact
about i fear cannot be a spare 		OUTPUT: ieee_be_part_who
i always want to collaborate 		OUTPUT: ieee_collab
where are the user selection initial criterias for being bell 		OUTPUT: ieee_be_part_who
what i even could ieee 		OUTPUT: ieee_what
can we all collaborate with you 		OUTPUT: ieee_collab
so i want immediate where can anybody yo

where are you in need bank sponsors 		OUTPUT: ieee_sponsor
if all can join ieee 		OUTPUT: ieee_be_part_who
wanna give me an overview of the domains you should 		OUTPUT: ieee_domains
what kind of stuff you guys do 		OUTPUT: ieee_projects
what became this original website 		OUTPUT: ieee_what
imagine what today is ieee 		OUTPUT: ieee_what
i would like to consider discuss some major sponsorship proposals 		OUTPUT: ieee_sponsor
what do they make at ieee 		OUTPUT: ieee_projects
can we now collaborate with ieee or possibly do an 		OUTPUT: ieee_collab
your media forum pages 		OUTPUT: ieee_social_media
if i wanted to ask her 		OUTPUT: gen_query
list of board executive members 		OUTPUT: ieee_current_board
where u want 		OUTPUT: gen_bot
what sorta boring events have yall ever done in such 		OUTPUT: ieee_past_events
explore what are techloops 		OUTPUT: ieee_techloop
where users could currently everyone find true github or repos 		OUTPUT: ieee_github
nobody can and i get that of ur social media 		O

what cool kinds football league games do you guys do 		OUTPUT: ieee_domains
thank love yo bye 		OUTPUT: ieee_how_long
but what ieee signal technology is coming 		OUTPUT: ieee_future_events
where fuck can i just hit you up on social 		OUTPUT: ieee_social_media
source server is ieee 		OUTPUT: ieee_what
bring me yo bout onto the projects ieee does 		OUTPUT: ieee_projects
oh hey u domains do u n guys do 		OUTPUT: ieee_domains
boo boo bot 		OUTPUT: gen_bot
and when was ieee different 		OUTPUT: ieee_how_long
can i u help 		OUTPUT: gen_help
what tests were 		OUTPUT: ieee_what
past three events you conducted 		OUTPUT: ieee_past_events
or is there a way i can eventually be made 		OUTPUT: ieee_be_part_who
where can i safely follow 		OUTPUT: ieee_social_media
software like githup or repos does ieee à¥¤ 		OUTPUT: ieee_github
what are ieee'hash hash functions 		OUTPUT: ieee_what_does
what do you nice guys be 		OUTPUT: ieee_what_does
how do i generally contact ieee on social network 		OUTPUT: ieee_s

more extensive questions 		OUTPUT: gen_human
tell me about the two past events 		OUTPUT: ieee_past_events
is ieee - compliant wie world definition 0 		OUTPUT: ieee_other_linked
got you serious 		OUTPUT: gen_how_you
celebrate your 60th 		OUTPUT: ieee_how_long
know what software projects do we do 		OUTPUT: ieee_projects
we do wanna sing with you people 		OUTPUT: ieee_collab
what all great events does ieee can 		OUTPUT: ieee_past_events
got a question card on you 		OUTPUT: gen_query
why can anyone collaborate with a publishing 		OUTPUT: ieee_collab
bye bye 		OUTPUT: ieee_domains
how can they connect with are you 		OUTPUT: ieee_further_contact
who do i make contact after further internal 		OUTPUT: ieee_further_contact
so what exciting are the possible upcoming events that you'll 		OUTPUT: ieee_future_events
who are are senior heads of ieee 		OUTPUT: ieee_current_board
use your my help 		OUTPUT: gen_help
a thanks 		OUTPUT: gen_thanks
are made other ieee implementations are way directly link

In [48]:
net_1.eval()
val_h = net_1.init_hidden(batch_size)
val_losses = []
a=[]
net_1.eval()
for inputs, labels in valid_loader:
        val_h = tuple([each.data for each in val_h])

        if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()
                inputs=torch.tensor(inputs).to(torch.int64)
                labels=torch.tensor(labels).to(torch.long)
                
        output, val_h = net_1(inputs, val_h)
        _,index=torch.topk(output,1)
        a.append(index.item()==labels.item())
        val_loss = criterion(output, labels)
        for i in inputs.squeeze():
            if i:
                print(int_to_word[i.item()],end=" ")
        print('OUTPUT:',int_to_label[index.item()])
                
        
        
        val_losses.append(val_loss.item())
print( "Val Loss: {:.6f}".format(np.mean(val_losses)),"        Val_Acuuracy:",(a.count(1)/(a.count(0)+a.count(1)))*100)

i have the question OUTPUT: gen_query
what could you do OUTPUT: ieee_projects
could you please help make me play out OUTPUT: gen_help
i want them watch what ieee

C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Guru Aathavan AL U\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


 OUTPUT: ieee_what_does
that particular guy named harsh OUTPUT: gen_who_made
what do you guys be OUTPUT: ieee_what_does
do you help OUTPUT: gen_help
how are the ieees of those other rival colleges linked OUTPUT: ieee_other_college
what defines exactly that ieee does OUTPUT: ieee_projects
which reflects the major events hosted by you OUTPUT: ieee_past_events
what data is ieee OUTPUT: ieee_what
i have some silly doubts OUTPUT: gen_query
what must ieee OUTPUT: ieee_what_does
are different ieees closely linked OUTPUT: ieee_other_college
are ieee - 400 nodes ieee OUTPUT: ieee_other_linked
matthew wassup OUTPUT: gen_how_you
is it any implications for being officially part of OUTPUT: ieee_be_part_who
i am no thank thank OUTPUT: gen_how_you
how doing you OUTPUT: gen_how_you
there are many ieee in what vit are suppose they OUTPUT: ieee_other_linked
so do i get eye touch with OUTPUT: ieee_further_contact
perhaps please help OUTPUT: gen_help
how ieee of residential colleges are linked in neighbou

are you a robot robot OUTPUT: gen_bot
what data is ieee OUTPUT: ieee_what
when all ieee OUTPUT: ieee_how_long
great luck to see OUTPUT: gen_hi
do you have another github repo where you may see OUTPUT: ieee_github
when does ieee have different OUTPUT: ieee_other_linked
who am i truly talking OUTPUT: gen_who_you
does ieee do it OUTPUT: ieee_what_does
what do how they do OUTPUT: ieee_what_does
for how much long what has ieee been OUTPUT: ieee_how_long
what isn kind one of projects does ieee do OUTPUT: ieee_projects
what sounds techloop OUTPUT: ieee_techloop
had existed so countless ieees in possibly all of these OUTPUT: ieee_other_linked
what all four domains does your chapter one OUTPUT: ieee_domains
i have no doubt OUTPUT: gen_query
i wanted to ask her OUTPUT: gen_query
or whatup OUTPUT: gen_hi
what our the ways in question ieee would sponsor OUTPUT: ieee_sponsor
what tech bits do you do OUTPUT: ieee_domains
what all events can reasonably expect in its future OUTPUT: ieee_future_events


are you in the midst of any help OUTPUT: ieee_sponsor
what kinds football games do you guys do OUTPUT: ieee_domains
i have what question to ask OUTPUT: gen_query
are other ieees in vit right for you in any OUTPUT: ieee_other_linked
have you i met you OUTPUT: gen_who_you
appreciate your OUTPUT: gen_thanks
what video does ieee do OUTPUT: ieee_what_does
what is ieee - wie formally part of OUTPUT: ieee_other_linked
explain internet ieee OUTPUT: ieee_what
show me 3 board members OUTPUT: ieee_current_board
contact you on social media OUTPUT: ieee_social_media
ee heyo OUTPUT: gen_hi
see what's up OUTPUT: gen_how_you
how shall we recognize you in future OUTPUT: ieee_further_contact
i finally have a question OUTPUT: gen_query
source is ieee OUTPUT: ieee_what
github email links OUTPUT: ieee_github
are you becoming a man made OUTPUT: gen_bot
so may you i know the identity of you'd re OUTPUT: gen_who_made
what jobs do you do at ieee OUTPUT: ieee_what_does
what are ieee'hash functions OUTPUT: ieee_

how old these are you OUTPUT: ieee_how_long
what kinda stuff does ieee really do OUTPUT: ieee_projects
what engineering domains does ieee cover OUTPUT: ieee_domains
move ahead OUTPUT: gen_query
is there sudden luck that is OUTPUT: ieee_future_events
ieee was ieee created OUTPUT: ieee_how_long
whats always your OUTPUT: gen_who_you
what are the projects parties are participating within OUTPUT: ieee_projects
are reminding you a person OUTPUT: gen_human
how accurate is OUTPUT: ieee_how_long
now all events have been OUTPUT: ieee_past_events
is sending data ieees of vit connected to its OUTPUT: ieee_other_linked
to want to hear something about ieee OUTPUT: ieee_what
miss are a aren'u t OUTPUT: gen_query
know you using any social OUTPUT: ieee_social_media
and how have you OUTPUT: gen_how_you
how long has ieee already been around OUTPUT: ieee_how_long
what exactly do you do OUTPUT: ieee_projects
are there other ieee chapters indexed in vit linked to OUTPUT: ieee_other_linked
what events has ie

In [115]:
torch.save(net_1.state_dict(),"ieeesite_v2.pt")

In [118]:
import pickle

In [ ]:
f=open("word_to_int.pkl","wb")
pickle.dump(word_to_int,f)
f.close()
f=open("int_to_word.pkl","wb")
pickle.dump(int_to_word,f)
f.close()

In [62]:
f=open("int_to_label.pkl","wb")
pickle.dump(int_to_label,f)
f.close()
f=open("label_to_int.pkl","wb")
pickle.dump(label_to_int,f)
f.close()